# Wordle Tweet Dataset


In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np
import re
import emoji
import scipy.stats as stats
import sklearn

Cominciamo importando il Dataset e visualizziamo alcune informazioni sulla tabella

In [2]:
wordle = pd.read_csv("WordleMegaData.csv")
wordle.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2140196 entries, 0 to 2140195
Data columns (total 10 columns):
 #   Column           Dtype  
---  ------           -----  
 0   WordleID         int64  
 1   ID               int64  
 2   Created_At       object 
 3   Text             object 
 4   Source           object 
 5   UserID           float64
 6   Username         object 
 7   User_ScreenName  object 
 8   Location         object 
 9   Truncated        bool   
dtypes: bool(1), float64(1), int64(2), object(6)
memory usage: 149.0+ MB


Andiamo ora a visualizzare le prime occorrenze della tabella

In [3]:
wordle.head()

,WordleID,ID,Created_At,Text,Source,UserID,Username,User_ScreenName,Location,Truncated
0,254,1498447921448034305,2022-02-28 23:59:58+00:00,Wordle 254 3/6\n\n🟨🟨⬛⬛⬛\n🟨⬛🟨⬛🟩\n🟩🟩🟩🟩🟩,Twitter for iPhone,3.681676e+07,Leslie Brown,live_laugh_pray,Haida Gwaii,False
1,254,1498447918184996864,2022-02-28 23:59:58+00:00,Wordle 254 4/6\n\n⬛⬛⬛⬛🟩\n⬛⬛⬛⬛🟩\n⬛🟩⬛⬛🟩\n🟩🟩🟩🟩🟩\n...,Twitter for iPhone,4.825910e+08,Dylan Sobo,DaBolt727,"Largo, FL / Bradenton|FGCU",False
2,254,1498447910173921282,2022-02-28 23:59:56+00:00,Wordle 254 3/6\n\n⬛⬛⬛⬛🟩\n⬛⬛🟩⬛🟩\n🟩🟩🟩🟩🟩,Twitter for iPhone,1.397624e+18,🖤,wengojos,23 • she/her • 🇵🇭🇺🇸,False
3,254,1498447901797801989,2022-02-28 23:59:54+00:00,Wordle 254 3/6\n\n⬛⬛⬛🟨⬛\n⬛🟨⬛⬛🟩\n🟩🟩🟩🟩🟩,Twitter for iPhone,1.255905e+18,Dustin Waters,DustinWaters12,NaN,False
4,254,1498447896911527938,2022-02-28 23:59:53+00:00,Wordle 254 3/6\n\n⬛🟨⬛⬛⬛\n🟩🟩🟩⬛🟩\n🟩🟩🟩🟩🟩,Twitter for iPhone,2.528937e+08,R. Colin,rollin_fatty,"Richmond, VA",False


## Pulizia dei dati

Per ogni carattersitica andiamo a visualizzare il numero di elementi nulli.

In [4]:
wordle.isnull().sum()

WordleID                0
ID                      0
Created_At              0
Text                    0
Source                  0
UserID                  0
Username              112
User_ScreenName         0
Location           573373
Truncated               0
dtype: int64

La caratteristica 'ID' rappresenta l'ID del tweet e quindi deve essere unica. Verifichiamo che sia così.

In [6]:
num_unique_elements = wordle['ID'].nunique()
total_elements = len(wordle['ID'])

print("Number of rows where 'ID' is unique:", num_unique_elements)
print("Total number of rows:", total_elements)

Number of rows where 'ID' is unique: 2140187
Total number of rows: 2140196


Sono presenti dei duplicati, li eliminiamo.

In [7]:
wordle = wordle.drop_duplicates(subset=['ID'], keep=False)
wordle.shape

(2140178, 10)

Ora che le nostre occorrenze sono uniche, andiamo a effettuare una pulizia del testo, mantenendo solamente le informazioni riguardanti la partita a wordle. 
Inoltre dal testo generiamo una colonna "Attempts" che rappresenta il numero di tentativi effettuati per indovinare la parola o la X nel caso di fallimento.

In [8]:
number_pattern = re.compile(r'(Wordle) (\d{3}) (([1-6]|X)\/6)')

# Function to extract the number of attempts from each row
def extract_attempts(row):
    match = number_pattern.search(row)
    if match:
        return match.group(3).replace('/6', '') 
    else:
        return np.nan

# Create a new column with the extracted numbers
wordle['Attempts'] = wordle['Text'].apply(extract_attempts)

wordle.head()

,WordleID,ID,Created_At,Text,Source,UserID,Username,User_ScreenName,Location,Truncated,Attempts
0,254,1498447921448034305,2022-02-28 23:59:58+00:00,Wordle 254 3/6\n\n🟨🟨⬛⬛⬛\n🟨⬛🟨⬛🟩\n🟩🟩🟩🟩🟩,Twitter for iPhone,3.681676e+07,Leslie Brown,live_laugh_pray,Haida Gwaii,False,3
1,254,1498447918184996864,2022-02-28 23:59:58+00:00,Wordle 254 4/6\n\n⬛⬛⬛⬛🟩\n⬛⬛⬛⬛🟩\n⬛🟩⬛⬛🟩\n🟩🟩🟩🟩🟩\n...,Twitter for iPhone,4.825910e+08,Dylan Sobo,DaBolt727,"Largo, FL / Bradenton|FGCU",False,4
2,254,1498447910173921282,2022-02-28 23:59:56+00:00,Wordle 254 3/6\n\n⬛⬛⬛⬛🟩\n⬛⬛🟩⬛🟩\n🟩🟩🟩🟩🟩,Twitter for iPhone,1.397624e+18,🖤,wengojos,23 • she/her • 🇵🇭🇺🇸,False,3
3,254,1498447901797801989,2022-02-28 23:59:54+00:00,Wordle 254 3/6\n\n⬛⬛⬛🟨⬛\n⬛🟨⬛⬛🟩\n🟩🟩🟩🟩🟩,Twitter for iPhone,1.255905e+18,Dustin Waters,DustinWaters12,NaN,False,3
4,254,1498447896911527938,2022-02-28 23:59:53+00:00,Wordle 254 3/6\n\n⬛🟨⬛⬛⬛\n🟩🟩🟩⬛🟩\n🟩🟩🟩🟩🟩,Twitter for iPhone,2.528937e+08,R. Colin,rollin_fatty,"Richmond, VA",False,3


Per estrarre ulteriori informazioni possiamo affidarci alle emoji riportate nei tweet, vi sono una serie di rappresentazioni alternative per la stessa informazione. Inoltre possiamo fare una verifica incrociata con il numero di tentativi per verificare la coerenza delle informazioni.

Le informazioni per le varie emoji sono le seguenti:
- ⬛ o ⬜ indicano che la lettera non è presente nella parola
- 🟦 o 🟨 indicano che la lettera è corretta ma non la sua posizione
- 🟩 o 🟧 indicano che la lettera è corretta e anche la sua posizione

*Possiamo sfruttare questa informazione per fare ulteriori analisi e/o test statistici, come per esempio, in media quanta gente indovina dopo una certa quantità di quadratini gialli/blu?*

Eliminiamo le righe che contengono nella colonna Text più occorrenze di una partita perchè ambigue. Quale delle partite nel tweet è stata effettivamente giocata? Quale corrisponde alla partita di oggi? 

In [11]:
colored_squares_pattern = re.compile(r'((([🟨🟩⬛⬜]{5})|([🟧🟦⬛⬜]{5})){6})', flags=re.UNICODE)

# Define the modified regex pattern
colored_squares_pattern_win = re.compile(r'((([🟨🟩⬛⬜]{5})|([🟧🟦⬛⬜]{5})){1,5}([🟩🟧]{5}))', flags=re.UNICODE)

text_array = ['🟨🟨⬛⬛⬛', '🟨⬛🟨⬛🟩', '🟩🟩🟩🟩🟩']

# Initialize an empty set to store unique matches
unique_matches = set()

# Loop through each text in the array and find matches
for text in text_array:
    unique_matches.update(colored_squares_pattern.findall(text))

print("Number of unique matches:", len(unique_matches))

Number of unique matches: 3


In [ ]:
colored_squares_pattern = re.compile(r'((([🟨🟩⬛⬜]{5})|([🟧🟦⬛⬜]{5})){1,6})', flags=re.UNICODE)
wordle['Text'] = wordle['Text'].astype(str)

# Function to extract emojis from a text
def extract_emojis(text):
    matches = colored_squares_pattern.findall(text)
    if not matches:
        return np.nan
    return [match[0] for match in matches]

# Apply the function to the 'Text' column
wordle['Text'] = wordle['Text'].apply(extract_emojis)

wordle.head()

L'informazione più affidabile su cui basarsi per identificare un utente all'interno del dataset è UserID. L'assenza di valori nulli per questo attributo ci fornisce maggiore sicurezza. Possiamo allora fare a meno delle colonne Username e User_ScreenName che possono contenere identificativi non unici e modificabili da parte dell'utente nel tempo.

Notiamo inoltre che la colonna Location contiene anche valori nulli e valori errati. Come ulteriore verifica, su Kaggle ci viene suggerito che il 27% dei valori è nullo, il 72% non viene associato a una macrocategoria, mentre l'1% dei tweet risulta provenire dagli Stati Uniti. Dopo aver verificato a nostra volta che il numero di valori nulli per questa caratteristica è elevato, eliminiamo la colonna. 

In [ ]:
wordle = wordle[wordle['Truncated'] != True]
wordle.drop(columns=['Username', 'User_ScreenName', 'Location', 'Truncated'], inplace=True)

Nel nostro dataset a ogni X corrisponde una partita persa, quindi a 6 tentativi. 

In [ ]:
wordle.isnull().sum()

In [ ]:
#wordle['Win'] = ~wordle['Attempts'].isin(['X', np.nan])
wordle['Attempts'] = wordle['Attempts'].replace('X', '6')

wordle['Text'] = wordle['Text'].astype(str)
wordle['Text'] = wordle['Text'].apply(lambda x: [x[i:i+5] for i in range(0, len(x), 5)])

# Print the first few rows to check the result
wordle.head()

In [ ]:
wordle.isnull().sum()

Riempiamo i valori nulli di Attempts.

In [ ]:
wordle.loc[wordle['Attempts'].isna(), 'Attempts'] = wordle.loc[wordle['Attempts'].isna(), 'Text'].apply(lambda x: len(x))

Verifichiamo che i nostri dati siano coerenti. Per prima cosa verifichiamo che per ogni campione il numero di gruppi da 5 quadrati consecutivi appena estratto sia uguale al valore contenuto nella caratteristica 'Attempts'. 

In [ ]:
def verify_length(row):
    # Get the length of the 'Text' array
    text_length = len(row['Text'])
    
    # Get the value in the 'Attempts' column
    attempts = row['Attempts']
    
    # Check if the lengths match
    return text_length != attempts

# Count the number of rows where lengths do not match
num_mismatched_rows = wordle.apply(verify_length, axis=1).sum()

print("Number of rows where the number of elements in 'Text' is not equal to 'Attempts':", num_mismatched_rows)

Eliminiamo le righe dove c'è incoerenza. 

In [ ]:
# Create a boolean mask indicating whether the number of elements in 'Text' matches 'Attempts'
mask = wordle.apply(lambda row: len(row['Text']) == row['Attempts'], axis=1)

# Drop the rows where the number of elements in 'Text' does not match 'Attempts'
wordle = wordle[mask]

# Now drop the 'Verify_Length' column
wordle.drop(columns=['Verify_Length'], inplace=True)

# Print the resulting DataFrame
wordle.head()

Ora che sappiamo che il numero di gruppi di quadrati consecutivi coincide sempre col numero di tentativi, continuiamo la verifica di coerenza, eliminando tutte quelle righe che contengono valori sbagliati del campo 'Text' (come ad esempio 6 righe di 5 quadrati tutti verdi). 

In [ ]:
def meets_conditions(row):
    # Check if any element in 'Text' contains five 🟩 or five 🟧 emojis
    for i, text in enumerate(row['Text']):
        if text.count('🟩') == 5 or text.count('🟧') == 5:
            # Check if it's the last element of the array
            if i == len(row['Text']) - 1:
                # If the condition is met and it's the last element, check if it's the only one
                num_matching_elements = sum(1 for t in row['Text'] if t.count('🟩') == 5 or t.count('🟧') == 5)
                if num_matching_elements == 1:
                    return True
                else:
                    return False
            else:
                return False
    # If no element contains five 🟩 or five 🟧 emojis, return True
    return True

wrong_rows = wordle[~wordle.apply(meets_conditions, axis=1)]

# Apply the function to each row of the DataFrame to filter out rows that don't meet the conditions
wordle = wordle[wordle.apply(meets_conditions, axis=1)]


In [ ]:
wrong_rows.head()

Abbiamo effettivamente eliminato dei campioni errati.
Terminiamo l'analisi creando una colonna 'Win' che ci indichi se una partita è stata vinta o persa. 

In [ ]:
def verify_winning_rows(row):
    # Get the last element of the 'Text_Array'
    last_element = row['Text'][-1]
        
    # Check if the last element contains five 🟩 or five 🟧 emojis
    if last_element.count('🟩') == 5 or last_element.count('🟧') == 5:
            # Check if the last element is the only one containing five 🟩 or five 🟧 emojis
            num_matching_elements = sum(1 for text in row['Text'] if text.count('🟩') == 5 or text.count('🟧') == 5)
            return num_matching_elements == 1
    return False

wordle['Win'] = wordle.apply(verify_winning_rows, axis=1)
wordle.head()

In [ ]:
wordle.shape

Nella fase di pulizia e integrazione dei dati abbiamo perso circa 100000 campioni sui 2140196 campioni totali, quindi circa il 5% della totalità dei campioni.

## Visualizzazione dei dati

In [ ]:
#play_per_day is a pandas Series object.
play_per_day = wordle['WordleID'].value_counts()
print(play_per_day.sample(n=10))

#win_per_day is a pandas Series object.
win_per_day = wordle.groupby('WordleID')['Win'].sum()
print(win_per_day.sample(n=10))

#win_percentage is a pandas Series object.
#fill_value=0 indica che viene messo 0 se play_per_day è zero (per evitare una divisione per zero) ma nel nostro caso non si verifica mai una
#divisione per zero (ho già verificato e in win_percentage fill_value=0 non viene mai applicato)
win_percentage = (win_per_day.div(play_per_day, fill_value=0)).mul(100)
print(win_percentage.sample(n=10))

Le percentuali di vincita sembrano tutte concentrate nell'intervallo compreso tra 90% e 100%. Cerchiamo di comprendere se è davvero così.

In [ ]:
print(win_percentage.mean())
print(win_percentage.var())
print(win_percentage.mode())
print(win_percentage.median())